In [1]:
from KeywordSearch import loader, indexing, utils
import gc

c:\Users\10022\Documents\GitHub\ttds-group-project\KeywordSearch\loader.py:174: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (complete_counter % 500 is 0):
c:\Users\10022\Documents\GitHub\ttds-group-project\KeywordSearch\loader.py:183: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if (complete_counter % 500 is 0):
c:\Users\10022\Documents\GitHub\ttds-group-project\KeywordSearch\indexing.py:134: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if complete_counter % 5 is 0: gc.collect()


In [2]:
segment_size = 5000
part_size = 5000

# for i in range(13):
#     gc.collect()
#     print(f"\nPart {i:02d}", flush=True)
#     indexing.build_full_index(offset=i*part_size, k=part_size, batch_size=segment_size, index_type="inverted", prefix=f"part{i:02d}", unsafe_pickle=False)

In [3]:
# sizes = utils.measure_sizes()

In [4]:
# list(sizes.items())[-1]

In [5]:
# indexing.merge_index()

In [6]:
gc.collect()
index = loader.load_merged_index(save_merged=False)

1140 segments to load
Finished merging 1140 segments...
Garbage collection done
All done


In [7]:
gc.collect()
from pympler.asizeof import asizeof
index_size = asizeof(index)
mb = 1024 * 1024
gb = mb * 1024
gb_size = index_size // gb
mb_size = round((index_size % gb) / mb)
print(f"{gb_size}GB" + f" {mb_size}MB" if mb_size else '')

44GB 370MB


In [8]:
all_tokens = loader.load_token_vocab(k=-1)

In [9]:
import pickle
with open("valid_books.pkl", "rb") as f:
    _, _, valid_books = pickle.load(f)

In [10]:
len(all_tokens), len(index)

(5696023, 5696023)

In [11]:
from typing import Iterable

import numpy as np
import scipy.sparse
from tqdm.notebook import tqdm

def construct_bool_table(index: Iterable[dict]):
    table = scipy.sparse.dok_matrix((len(all_tokens), max(valid_books) + 1), dtype=np.bool_)
    print(len(index), table.shape)
    length = len(all_tokens)
    for token_id, token_dict in tqdm(enumerate(index[:length]), total=length):
        if token_dict:
            table[token_id, tuple(token_dict.keys())] = True
    gc.collect()
    return table.tocsc()

In [12]:
table = construct_bool_table(index)
scipy.sparse.save_npz("lookup_table.npz", table, compressed=True)

5696023 (5696023, 72533)


  0%|          | 0/5696023 [00:00<?, ?it/s]

In [16]:
(table.data.nbytes + table.indptr.nbytes + table.indices.nbytes) // 1024 // 1024

1063

In [17]:
dok_table: scipy.sparse.dok_matrix = table.todok()

In [20]:
asizeof(dok_table) // 1024 // 1024

In [ ]:
asizeof(table) // 1024 // 1024

In [ ]:
scipy.sparse.save_npz("lookup_table_coo.npz", table.tocoo(), compressed=True)

In [7]:
x = list(range(5696023))
y = []
index_size = len(x)
batch_size = 5000
batches = index_size // batch_size
for i in range(batches):
    start = i * batch_size
    end = min(start + batch_size, index_size)
    y.extend(x[start:end])

y.extend(x[end:])
x == y, end

(True, 5695000)

In [56]:
with open("all_tokens_old.pkl", 'rb') as f:
    _, _, all_tokens_old = pickle.load(f)
len(all_tokens_old)

5696023

In [ ]:
# inverted_index_2, indexed_books_2 = indexing.build_full_index(offset=5000, k=5000, batch_size=-1, index_type="inverted", prefix="part02")
# import h5py
# def save_inv_index_HDF5(filename: str, index: list[dict], **kwargs):
#     with h5py.File(filename, 'w') as f:
#         for i, entry in enumerate(index):
#             group = f.create_group(str(i))
#             for book_id, occurrences in entry.items():
#                 group.create_dataset(str(book_id), data=occurrences, **kwargs)
# save_inv_index_HDF5("test.h5", inverted_index_1, chunks=True, compression="gzip")

In [ ]:
import pickle
import numpy as np

with open("index/inverted_113.pkl", "rb") as f:
    tmp = pickle.load(f)

In [ ]:
len(tmp)


In [ ]:
#bow_index, indexed_books = indexing.build_full_index(k=-1)

In [ ]:
import pickle
import numpy as np

with open("index.pkl", "rb") as f:
    bow_index = pickle.load(f)
# dummy_arr = np.array([], dtype=np.uint8)
# dummy = (0, dummy_arr, 0, dummy_arr)
# bow_index = tuple([dummy] + [pair if isinstance(pair, tuple) else dummy for pair in bow_index[1:]])
# with open("index.pkl", "wb") as f:
#     pickle.dump(bow_index, f)

with open("valid_books.pkl", "rb") as f:
    _, _, valid_books = pickle.load(f)

book_list = sorted(valid_books)

non_empty_books = []
non_empty_index = []
for book_id, book_bow in enumerate(bow_index):
    if book_bow:
        non_empty_books.append(book_id)
        non_empty_index.append(book_bow)

In [ ]:
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.similarities import MatrixSimilarity, SparseMatrixSimilarity
import concurrent.futures

In [ ]:
# corpus = [utils.deltazip([vocab, counts], [vocab_delta, count_delta]) for vocab_delta, vocab, count_delta, counts in non_empty_index]
# tfidf_model = TfidfModel(corpus, smartirs='ntc')
# with open("tfidf_model.pkl", "wb") as f:
#     pickle.dump(tfidf_model, f)
with open("tfidf_model.pkl", "rb") as f:
    tfidf_model = pickle.load(f)

In [ ]:
dim = len(all_tokens) + 1
def cast2numpy(x):
    arr = np.zeros(dim, dtype=np.float16)
    for i, score in x:
        arr[i] = score
    return arr

In [ ]:
tmp = [tfidf_model[utils.deltazip([vocab, counts], [vocab_delta, count_delta])] for vocab_delta, vocab, count_delta, counts in non_empty_index]
# with open("tfidf_index.pkl", "rb") as f:
#     tmp = pickle.load(f)

In [ ]:
tfidf_model[[(0,1),(3,8)]]

In [ ]:
vocab_delta, vocab, count_delta, counts = non_empty_index[0]
cast2numpy(tfidf_model[utils.deltazip([vocab, counts], [vocab_delta, count_delta])]).mean()

In [ ]:
dim = len(all_tokens) + 1

In [ ]:
m1 = SparseMatrixSimilarity(tmp, num_best=10, num_features=dim)

In [ ]:
with open("tfidf_index.pkl", "wb") as f:
    pickle.dump(tmp, f)
with open("matrix.pkl", "wb") as f:
    pickle.dump(m1, f)

In [ ]:
asizeof.asizeof(tmp)/1024/1024/1024

In [ ]:
asizeof.asizeof(m1)/1024/1024/1024

In [ ]:
vocab_delta, vocab, count_delta, counts = non_empty_index[269]
next(utils.deltazip([vocab, counts], [vocab_delta, count_delta]))

In [ ]:
non_empty_index[269]

In [ ]:
tokens = [1, 3, 1, 9, 3, 5654]
vocab_list = sorted(set(tokens))
token_arr = np.array(tokens)
vocab, vocab_delta = utils.cast2intarr(vocab_list)
counts, count_delta = utils.cast2intarr([np.sum(token_arr == token) for token in vocab_list])
(vocab_delta, vocab, count_delta, counts, vocab_list, [(vocab == token-vocab_delta) for token in vocab_list])
vocab + vocab_delta, counts + count_delta

In [ ]:
book_path_template = loader.token_dir + "PG%d_tokens.txt"

def read_tokens(PG_id: int):
    with open(book_path_template %PG_id, encoding="UTF-8", errors="ignore") as f:
        return loader.stemmer.stemWords(f.read().splitlines())

In [ ]:


docs = []
docs_index = []
failed_jobs = 0
complete_counter = 0
with concurrent.futures.ThreadPoolExecutor() as pool:
    jobs = {
        pool.submit(
            read_tokens, book_id)
            : book_id for book_id in indexed_books
        }
    
    for job in concurrent.futures.as_completed(jobs):
        book_id = jobs[job]
        try:
            result = job.result()
            docs.append(result)
            docs_index.append(book_id)
        except Exception as e:
            # raise e
            failed_jobs.append(book_id)
        complete_counter += 1
        print(f"Finished fetching tokens in {complete_counter} books...", end="\r")
        
        print(f"\n{len(failed_jobs)}/{len(jobs)} token fetching jobs failed")

In [ ]:
dictionary = Dictionary(docs)
corpus = [dictionary.doc2bow(doc) for doc in docs]
del docs
tfidf = TfidfModel(corpus, smartirs='ntc')
tfidf_corpus = [tfidf[doc] for doc in corpus]
del corpus, tfidf
index = MatrixSimilarity(tfidf_corpus, num_features=len(dictionary))

In [ ]:
# import pickle
# with open("all_tokens.pkl", 'rb') as f:
#     k, offset, _all_tokens = pickle.load(f)
# with open("all_tokens.pkl", "wb") as f:
#     pickle.dump((k, offset, tuple(_all_tokens)), f)

In [ ]:
from pympler import asizeof
asizeof.asizeof(bow_index), asizeof.asizeof(index)

In [ ]:
415963552/1024/1024

In [ ]:
415971552/1024/1024

In [ ]:

mem_size_lst = asizeof.asizeof(all_tokens)
mem_size_tup = asizeof.asizeof(tuple(all_tokens))
mem_size_lst/1024/1024, mem_size_tup

In [ ]:
from math import log2
import numpy as np
log2(len(all_tokens))
x = np.array([1,2,3])
x.shape[0]

In [ ]:
#loader.build_full_index()

In [ ]:
import pickle

In [ ]:
with open("index.pkl", "rb") as f:
    index = pickle.load(f)

In [ ]:
from pympler import asizeof
mem_size = asizeof.asizeof(index)

In [ ]:
mem_size/1024/1024

In [ ]:
index[1]

In [ ]:
size = 0
print(len(index))
for i, token in enumerate(index):
    for arr in token.values():
        size += asizeof.asizeof(arr)
    print(i, end='\r')
size

In [ ]:
size/1024/1024